======================================================================================================
PROBLEM A3

Build a classifier for the Human or Horse Dataset with Transfer Learning.
The test will expect it to classify binary classes.
Note that all the layers in the pre-trained model are non-trainable.
Do not use lambda layers in your model.

The horse-or-human dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
Inception_v3, pre-trained model used in this problem is developed by Google.

Desired accuracy and validation_accuracy > 97%.
=======================================================================================================


In [1]:
import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
local_file = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/horse-or-human')
zip_ref.close()

data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
local_file = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/validation-horse-or-human')
zip_ref.close()

In [2]:
train_dir = 'data/horse-or-human'
validation_dir = 'data/validation-horse-or-human'

train_datagen = ImageDataGenerator(
    # YOUR CODE HERE
    rescale=1./255.,
    rotation_range= 40,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True
)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    batch_size=32,
    class_mode='binary',
    target_size=(150,150))

# YOUR IMAGE SIZE SHOULD BE 150x150
# YOUR CODE HERE
validation_datagen=  ImageDataGenerator(
    rescale=1./255.,
    rotation_range= 40,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True
)

validation_generator = validation_datagen.flow_from_directory(
    directory=validation_dir,
    batch_size=32,
    class_mode='binary',
    target_size= (150, 150)
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [3]:
inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
urllib.request.urlretrieve(
    inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [4]:
def create_pre_trained_model(local_weights_file):
  pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                  include_top = False,
                                  weights = None)

  pre_trained_model.load_weights(local_weights_file)

  for layer in pre_trained_model.layers:
    layer.trainable = False

  return pre_trained_model

In [5]:
pre_trained_model = create_pre_trained_model(local_weights_file)

In [6]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.98):
      print("\nReached 98% accuracy so cancelling training!")
      self.model.stop_training = True

In [7]:
last_layer = last_layer = pre_trained_model.get_layer('mixed7')

In [8]:
x = layers.Flatten()(last_layer.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [9]:
callbacks = myCallback()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 100,
                    callbacks=callbacks)

Epoch 1/100
33/33 [==============================] - 24s 485ms/step - loss: 0.1943 - accuracy: 0.9299 - val_loss: 0.2819 - val_accuracy: 0.9023
Epoch 2/100
33/33 [==============================] - ETA: 0s - loss: 0.0370 - accuracy: 0.9873
Reached 98% accuracy so cancelling training!
33/33 [==============================] - 12s 359ms/step - loss: 0.0370 - accuracy: 0.9873 - val_loss: 0.0531 - val_accuracy: 0.9844


In [10]:
model.save("model_A3.h5")